# Network Improvements
---
Use this section to add new links or improve existing ones. New links should be drawn in a GIS program. Functions for adding reference columns and splitting existing links will be added in the future, but need to be manually added for now. If wanting to see the impacts of only one improvement/new link, be sure to export that as a separate network.

In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

import networkx as nx
from tqdm import tqdm
import pickle

from bikewaysim.paths import config
from bikewaysim.network import matching_script

# Settings

In [ ]:
buffer_ft = 100
max_hausdorff_dist = 500

# Import network

In [ ]:
links = gpd.read_file(config['network_fp'] / 'networks.gpkg',layer='osm_links')

# filter to roads
links = links[links['link_type']=='road']

#add street names back to osm
#add attributes back (especially the oneway column)
osm_attrs = gpd.read_file(config['osmdwnld_fp'] / f"osm.gpkg",layer='raw',ignore_geometry=True)
cols_to_keep = ['osmid','highway','name']
links = pd.merge(links,osm_attrs[cols_to_keep],on='osmid')
del osm_attrs

# GDOT Study Area

In [ ]:
other_name = "coa"

In [ ]:
# import improvements
improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer=config['mod_name'])
improvements

In [ ]:
improvements = improvements[[f"{config['mod_name']}_id",f"{config['mod_name']}_name",f"{config['mod_name']}_osm_type","geometry"]]

In [ ]:
# copy to prevent modification
links_buffered = links.copy()
other_source = improvements.copy()

# buffer the osm cycleways
links_buffered.geometry = links_buffered.buffer(buffer_ft)

# intersect with coa/arc (returns coa/arc linestrings)
overlap = gpd.overlay(other_source,links_buffered)

#street name check if for bike lanes / sharrows / cycletracks
#TODO use the new method for this step
overlap['name'] = overlap['name'].apply(lambda row: matching_script.remove_suffix(row))
overlap[f"{other_name}_name0"] = overlap[f"{other_name}_name"].apply(lambda row: matching_script.remove_suffix(row))
overlap['name_check'] = overlap.apply(lambda row: matching_script.name_check(row['name'],row[f"{other_name}_name0"]),axis=1)

In [ ]:
# only accept matching names
overlap = overlap[(overlap['name_check']==True)]

In [ ]:
overlap.drop_duplicates(inplace=True)

In [ ]:
### HAUSDORFF DISTANCE CHECK ###
# add osm geometry to compare against arc/coa geometry
overlap = pd.merge(overlap,links[['osm_linkid','geometry']],on='osm_linkid')
overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
overlap.drop(columns=['geometry_x'],inplace=True)
overlap.rename(columns={'geometry_y':'geometry'},inplace=True)

# replace intersected geometry with the original geometry
overlap = gpd.GeoDataFrame(overlap,geometry='geometry')

In [ ]:
# for remaining multi matches choose match with the lowest hausdorff distance
min_hausdorff = overlap.groupby('osm_linkid')['hausdorff_dist'].idxmin()
overlap = overlap.loc[min_hausdorff]

## Visually, it looks like most of the improvements were properly assigned

In [ ]:
improvements_buffer = improvements.copy()
improvements_buffer.geometry = improvements_buffer.buffer(200)
m = improvements_buffer.explore(f'{other_name}_osm_type')
overlap[['osm_linkid',f'{other_name}_osm_type','coa_name','name','name_check','hausdorff_dist','geometry']].explore(color='red',m=m)#.sort_values('hausdorff_dist')

## Clean Up

In [ ]:
needed_cols = [f'{other_name}_id', f'{other_name}_name', f'{other_name}_osm_type', 'osm_linkid', 'geometry']

In [ ]:
print(overlap['coa_osm_type'].unique())
overlap.loc[overlap['coa_osm_type']=='buffered bike lane','coa_osm_type'] = 'bike lane'
print(overlap['coa_osm_type'].unique())

In [ ]:
overlap = overlap[needed_cols]
overlap.rename(columns={f'{other_name}_osm_type':'improvement'},inplace=True)
overlap.to_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer='coa')

In [ ]:
improvements.geometry.apply(lambda x: x.type).value_counts()
improvements = improvements[improvements.geometry.apply(lambda x: x.type)!='MultiLineString']

In [ ]:
from shapely.ops import Point
improvements = improvements[improvements['coa_id'].isin(overlap['coa_id'])]
starts = improvements['geometry'].apply(lambda x: Point(list(x.coords)[0]))
ends = improvements['geometry'].apply(lambda x: Point(list(x.coords)[-1]))
starts_and_ends = gpd.GeoDataFrame({'geometry':pd.concat([starts,ends])},crs=config['projected_crs_epsg'])
starts_and_ends.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='improvements_start_end')
starts_and_ends.explore()

In [ ]:
# improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer=config['mod_name'])

In [ ]:
# improvements[improvements['coa_id'].isin(overlap['coa_id'])]

In [ ]:
# Savannah Study Area 
# buffer_ft = 100
# max_hausdorff_dist = 500
# other_name = 'savannah'
# # import improvements
# improvements = gpd.read_file(config['bicycle_facilities_fp']/"network_modifications.gpkg",layer='savannah')
# # copy to prevent modification
# links_buffered = links.copy()
# other_source = improvements.copy()

# # buffer the osm cycleways
# links_buffered.geometry = links_buffered.buffer(buffer_ft)

# # intersect with coa/arc (returns coa/arc linestrings)
# overlap = gpd.overlay(other_source,links_buffered)

# #street name check if for bike lanes / sharrows / cycletracks
# overlap['name'] = overlap['name'].apply(lambda row: matching_script.remove_suffix(row))
# overlap[f"{other_name}_name"] = overlap[f"{other_name}_name"].apply(lambda row: matching_script.remove_suffix(row))
# overlap['name_check'] = overlap.apply(lambda row: matching_script.name_check(row['name'],row[f"{other_name}_name"]),axis=1)
# # only accept matching names
# overlap = overlap[(overlap['name_check']==True)]
# overlap.drop_duplicates(inplace=True)
# ### HAUSDORFF DISTANCE CHECK ###
# # add osm geometry to compare against arc/coa geometry
# overlap = pd.merge(overlap,links[['osm_linkid','geometry']],on='osm_linkid')
# overlap['hausdorff_dist'] = overlap.apply(lambda row: row['geometry_x'].hausdorff_distance(row['geometry_y']),axis=1)
# overlap.drop(columns=['geometry_x'],inplace=True)
# overlap.rename(columns={'geometry_y':'geometry'},inplace=True)

# # replace intersected geometry with the original geometry
# overlap = gpd.GeoDataFrame(overlap,geometry='geometry')
# # for remaining multi matches choose match with the lowest hausdorff distance
# min_hausdorff = overlap.groupby('osm_linkid')['hausdorff_dist'].idxmin()
# overlap = overlap.loc[min_hausdorff]
# ## Visually, it looks like most of the improvements were properly assigned
# improvements_buffer = improvements.copy()
# improvements_buffer.geometry = improvements_buffer.buffer(200)
# m = improvements_buffer.explore(f'{other_name}_osm_type')
# overlap[[f'osm_linkid',f'{other_name}_osm_type',f'{other_name}_name','name','name_check','hausdorff_dist','geometry']].explore(color='red',m=m)#.sort_values('hausdorff_dist')
# ## Clean Up
# overlap
# needed_cols = [f'{other_name}_id', , f'{other_name}_osm_type', 'osm_linkid', 'geometry']
# # #replace
# # overlap[[f'{other_name}_osm_type','facility_fwd']].value_counts(dropna=False)
# # overlap.loc[(overlap[f'{other_name}_osm_type']=='cycletrack') & (overlap['facility_fwd']=='bike lane'),needed_cols+['facility_fwd']].explore(m=m)
# overlap = overlap[needed_cols]
# overlap.rename(columns={f'{other_name}_osm_type':'improvement'},inplace=True)
# overlap.to_file(config['bicycle_facilities_fp']/"network_improvements.gpkg",layer=f'{other_name}')
